# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_df = pd.read_csv("../Resources/weather_df.csv")
weather_df


,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Date
0,0,mar del plata,AR,-38.0023,-57.5575,7.0,81.0,46.94,4.00,1.652598e+09
1,1,castro,BR,-24.7911,-50.0119,100.0,98.0,54.59,2.89,1.652598e+09
2,2,taolanaro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,hilo,US,19.7297,-155.0900,77.0,98.0,72.28,1.99,1.652598e+09
4,4,rikitea,PF,-23.1203,-134.9692,100.0,72.0,75.83,20.85,1.652598e+09
...,...,...,...,...,...,...,...,...,...,...
631,631,nouadhibou,MR,20.9310,-17.0347,100.0,75.0,68.40,18.63,1.652599e+09
632,632,coatesville,US,39.9832,-75.8238,100.0,97.0,65.48,2.35,1.652598e+09
633,633,nantucket,US,41.2835,-70.0995,100.0,98.0,60.94,5.75,1.652599e+09
634,634,hastings,GB,50.8552,0.5729,100.0,82.0,60.30,12.55,1.652599e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity (%)"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity (%)"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
filtered_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp (F)"] < 80) & (filtered_weather_df["Max Temp (F)"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed (mph)"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness (%)"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Date
36,36,pineville,US,31.3224,-92.4343,0.0,84.0,75.09,0.00,1.652598e+09
172,172,ixtapa,MX,20.7000,-105.2000,0.0,88.0,75.22,1.01,1.652598e+09
183,183,saint george,US,37.1041,-113.5841,0.0,15.0,75.94,4.61,1.652598e+09
217,217,beloha,MG,-25.1667,45.0500,0.0,58.0,75.38,9.37,1.652598e+09
229,229,surt,LY,31.2089,16.5887,0.0,40.0,71.65,7.40,1.652598e+09
248,248,alice,US,27.7523,-98.0697,0.0,88.0,74.77,6.91,1.652598e+09
308,308,port hueneme,US,34.1478,-119.1951,0.0,62.0,75.18,3.44,1.652598e+09
313,313,stephenville,US,32.2207,-98.2023,0.0,74.0,74.16,6.91,1.652598e+09
388,388,jalu,LY,29.0331,21.5482,0.0,37.0,72.75,9.31,1.652598e+09
399,399,hervey bay,AU,-25.2986,152.8535,0.0,85.0,74.44,4.03,1.652598e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 36: pineville.
Closest hotel is Country Inn & Suites by Radisson, Pineville, LA.
------------
Retrieving Results for Index 172: ixtapa.
Closest hotel is hotel la terminal.
------------
Retrieving Results for Index 183: saint george.
Closest hotel is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 217: beloha.
Missing field/result... skipping.
------------
Retrieving Results for Index 229: surt.
Closest hotel is الهادي ابوشعاله سرت.
------------
Retrieving Results for Index 248: alice.
Closest hotel is Holiday Inn Express & Suites Alice, an IHG Hotel.
------------
Retrieving Results for Index 308: port hueneme.
Closest hotel is Hampton Inn Channel Islands Harbor.
------------
Retrieving Results for Index 313: stephenville.
Closest hotel is La Quinta Inn & Suites by Wyndham Stephenville.
------------
Retrieving Results for Index 388: jalu.
Closest hotel is صاحبة الفخامة.
------------
Retrieving Results for Index 399: hervey bay.
Closest

In [9]:
hotel_df

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Date,Hotel Name
36,36,pineville,US,31.3224,-92.4343,0.0,84.0,75.09,0.00,1.652598e+09,"Country Inn & Suites by Radisson, Pineville, LA"
172,172,ixtapa,MX,20.7000,-105.2000,0.0,88.0,75.22,1.01,1.652598e+09,hotel la terminal
183,183,saint george,US,37.1041,-113.5841,0.0,15.0,75.94,4.61,1.652598e+09,Best Western Plus Abbey Inn
217,217,beloha,MG,-25.1667,45.0500,0.0,58.0,75.38,9.37,1.652598e+09,NaN
229,229,surt,LY,31.2089,16.5887,0.0,40.0,71.65,7.40,1.652598e+09,الهادي ابوشعاله سرت
248,248,alice,US,27.7523,-98.0697,0.0,88.0,74.77,6.91,1.652598e+09,"Holiday Inn Express & Suites Alice, an IHG Hotel"
308,308,port hueneme,US,34.1478,-119.1951,0.0,62.0,75.18,3.44,1.652598e+09,Hampton Inn Channel Islands Harbor
313,313,stephenville,US,32.2207,-98.2023,0.0,74.0,74.16,6.91,1.652598e+09,La Quinta Inn & Suites by Wyndham Stephenville
388,388,jalu,LY,29.0331,21.5482,0.0,37.0,72.75,9.31,1.652598e+09,صاحبة الفخامة
399,399,hervey bay,AU,-25.2986,152.8535,0.0,85.0,74.44,4.03,1.652598e+09,Shelly Bay Resort


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)


In [12]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))